In [1]:
import os
import sys
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint


sys.path.insert(0, os.path.abspath(os.path.join('..')))

from src.dataset import TCRSeqDataModule
from src.models import LightningNetTCR
from src.utils import blosum50_full, blosum50_20aa

In [2]:
tcrseq = TCRSeqDataModule(path_to_file='../data/preprocessed/sample_train.csv',
                        path_to_test_file='../data/preprocessed/sample_test.csv',
                        peptide_len=9, cdra_len=30, cdrb_len=30)
tcrseq.setup(sep=',', encoding=blosum50_20aa)
train_loader = tcrseq.train_dataloader()
val_loader = tcrseq.val_dataloader()
test_loader = tcrseq.test_dataloader()

In [3]:
nettcr = LightningNetTCR(peptide_len=9, cdra_len=30, cdrb_len=30, batch_size=tcrseq.hparams.batch_size)
checkpoint_callback = ModelCheckpoint(dirpath=os.path.join('checkpoint','nettcr-data', 'nettcr', save_top_k=1, monitor='val_loss')
tb_logger = pl_loggers.TensorBoardLogger(save_dir=os.path.join('logs','nettcr-data'), name='nettcr')
trainer = pl.Trainer(max_epochs=20, logger=tb_logger, callbacks=[checkpoint_callback], log_every_n_steps=10)
trainer.fit(nettcr, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/jgbrasier/miniforge3/envs/geo_lightning/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1789: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
Missing logger folder: logs/nettcr/nettcr-data/lightning_logs

  | Name       | Type            | Params
-----------------------------------------------
0 | pep_conv   | ModuleList      | 3.7 K 
1 | cdra_conv  | ModuleList      | 12.1 K
2 | cdrb_conv  | ModuleList      | 12.1 K
3 | pool       | GlobalMaxPool1D | 0     
4 | activation | Sigmoid         | 0     
5 | lin1       | Linear          | 7.7 K 
6 | lin2       | Linear          | 33    
7 | criterion  | BCELoss         | 0     
-----------------------------------------------
35.6 K    Trainable params
0         Non-trainable para

Sanity Checking: 0it [00:00, ?it/s]

/Users/jgbrasier/miniforge3/envs/geo_lightning/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/jgbrasier/miniforge3/envs/geo_lightning/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/jgbrasier/miniforge3/envs/geo_lightning/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
%tensorboard --logdir logs